In [ ]:
%pip install azure-ai-ml
%pip install -U 'azureml-rag[faiss]>=0.1.14'

In [ ]:
# If `import win32file` fails with a DLL error then run the following and restart kernel:
# %pip uninstall -y pywin32
# %conda install -y --force-reinstall pywin32

# QA Data Generation

QA Data Generation is a part of RAG (Retrieval Augemented Generation) creation process where the autogenerated QA dataset is used:

1. To get the best prompt for RAG
2. To get evaluation metrics for RAG

This notebook shows you how to create a QA dataset from your data (Git repo). We run just the components needed for QA Data Generation and not for the full RAG creation flow.

## Get client for AzureML Workspace

The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

Enter your Workspace details below, running this still will write a `workspace.json` file to the current folder.

In [ ]:
%%writefile workspace.json
{
    "subscription_id": "<subscription_id>",
    "resource_group": "<resource_group_name>",
    "workspace_name": "<workspace_name>"
}

`MLClient` is how you interact with AzureML

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

## Azure OpenAI

We recommend using gpt-35-turbo model to get good quality QAs. [Follow these instructions](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) to setup an Azure OpenAI Instance and deploy the model. Once you have the model deployed in AOAI you can specify your Model name and Deployment name below.

We will use the automatically created `Default_AzureOpenAI` connection, change `aoai_connection_name` to use your own.

In [ ]:
aoai_connection_name = "Default_AzureOpenAI"

In [ ]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

try:
    aoai_connection = get_connection_by_name_v2(ws, aoai_connection_name)
except Exception as ex:
    # Create New Connection
    # Modify the details below to match the `Endpoint` and API key of your AOAI resource, these details can be found in Azure Portal
    raise RuntimeError(
        "Have you entered your AOAI resource details below? If so, delete me!"
    )
    aoai_connection = create_connection_v2(
        workspace=ws,
        name=aoai_connection,
        category="AzureOpenAI",
        # 'Endpoint' from Azure OpenAI resource overview
        target="https://<endpoint_name>.openai.azure.com/",
        auth_type="ApiKey",
        credentials={
            # Either `Key` from the `Keys and Endpoint` tab of your Azure OpenAI resource, will be stored in your Workspace associated Azure Key Vault.
            "key": "<api-key>"
        },
        metadata={"ApiType": "azure", "ApiVersion": "2023-05-15"},
    )

aoai_connection_id = aoai_connection["id"]

Now that your Workspace has a connection to Azure OpenAI we will make sure the `gpt-35-turbo` model has been deployed ready for inference. This cell will fail if there is not deployment for the embeddings model, [follow these instructions](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#deploy-a-model) to deploy a model with Azure OpenAI.

In [ ]:
from azureml.rag.utils.deployment import infer_deployment

aoai_completion_model_name = "gpt-35-turbo"

try:
    aoai_completion_deployment_name = infer_deployment(
        aoai_connection, aoai_completion_model_name
    )
except Exception as e:
    print(
        f"Deployment name in AOAI workspace for model '{aoai_completion_model_name}' is not found."
    )
    print(
        f"Please create a deployment for this model by following the deploy instructions on the resource page for '{aoai_connection['properties']['target']}' in Azure Portal."
    )

print(
    f"Deployment name in AOAI workspace for model '{aoai_completion_model_name}' is '{aoai_completion_deployment_name}'"
)

Finally we will combine the deployment and model information into a uri form which the AzureML embeddings components expect as input.

In [ ]:
llm_completion_config = f'{{"type":"azure_open_ai","model_name":"{aoai_completion_model_name}","deployment_name":"{aoai_completion_deployment_name}","temperature":0,"max_tokens":"1500"}}'

### Setup Pipeline

AzureML [Pipelines](https://learn.microsoft.com/azure/machine-learning/concept-ml-pipelines?view=azureml-api-2) connect together multiple [Components](https://learn.microsoft.com/azure/machine-learning/concept-component?view=azureml-api-2). Each Component defines inputs, code that consumes the inputs and outputs produced from the code. Pipelines themselves can have inputs, and outputs produced by connecting together individual sub Components.
To process your data for embedding and indexing we will chain together multiple components each performing their own step of the workflow.

The Components are published to a [Registry](https://learn.microsoft.com/azure/machine-learning/how-to-manage-registries?view=azureml-api-2&tabs=cli), `azureml`, which should have access to by default, it can be accessed from any Workspace.
In the below cell we get the Component Definitions from the `azureml` registry.

In [ ]:
ml_registry = MLClient(credential=credential, registry_name="azureml")

validate_deployments_component = ml_registry.components.get(
    "llm_rag_validate_deployments", label="latest"
)
# Clones git repository to output folder of pipeline, by default this will be on the default Workspace Datastore `workspaceblobstore`
git_clone_component = ml_registry.components.get("llm_rag_git_clone", label="latest")
# Walks input folder according to provided glob pattern (all files by default: '**/*') and attempts to open them, extract text chunks and further chunk if necessary to fir within provided `chunk_size`.
crack_and_chunk_component = ml_registry.components.get(
    "llm_rag_crack_and_chunk", label="latest"
)
# Generates test data for QA model training and evaluation
data_generation_component = ml_registry.components.get(
    "llm_rag_qa_data_generation", label="latest"
)
# Takes a uri to a storage location where test data is stored and registers it as a uri_file Data asset in the AzureML Workspace.
register_qa_data_component = ml_registry.components.get(
    "llm_rag_register_qa_data_asset", label="latest"
)

Each Component has documentation which provides an overall description of the Components purpose and each of the inputs/outputs.
For example we can see understand what `data_generation_component` does by inspecting the Component definition.

In [ ]:
print(data_generation_component)

Below a Pipeline is built by defining a python function which chains together the above components inputs and outputs. Arguments to the function are inputs to the Pipeline itself and the return value is a dictionary defining the outputs of the Pipeline.

In [ ]:
from azure.ai.ml import Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities._job.pipeline._io import PipelineInput
from typing import Optional


def use_automatic_compute(
    component, instance_count=1, instance_type="Standard_DS12_v2"
):
    """Configure input `component` to use automatic compute with `instance_count` and `instance_type`.

    This avoids the need to provision a compute cluster to run the component.
    """
    component.set_resources(
        instance_count=instance_count,
        instance_type=instance_type,
        properties={"compute_specification": {"automatic": True}},
    )
    return component


def optional_pipeline_input_provided(input: Optional[PipelineInput]):
    """Checks if optional pipeline inputs are provided."""
    return input is not None and input._data is not None


def use_aoai_connection(component, aoai_connection_id, custom_env: str = None):
    """Configure input `component` to use AOAI connection."""
    if optional_pipeline_input_provided(aoai_connection_id):
        if custom_env is not None:
            component.environment_variables[custom_env] = aoai_connection_id
        else:
            component.environment_variables[
                "AZUREML_WORKSPACE_CONNECTION_ID_AOAI"
            ] = aoai_connection_id


# If you have an existing compute cluster you want to use instead of automatic compute, uncomment the following line, replace `dedicated_cpu_compute` with the name of your cluster.
# Also comment out the `component.set_resources` line in `use_automatic_compute` above and the `default_compute='serverless'` line below.
# @pipeline(compute=dedicated_cpu_compute)
@pipeline(default_compute="serverless")
def qa_data_generation(
    git_url: str,
    asset_name: str,
    llm_completion_config: str,
    data_source_glob: str = None,
    data_source_url: str = None,
    document_path_replacement_regex: str = None,
    aoai_connection_id: str = None,
):
    """Pipeline for generating QA data from a git repository."""

    validate_deployments = validate_deployments_component(
        llm_config=llm_completion_config,
        check_completion="True",
        check_embeddings="False",
    )
    use_automatic_compute(validate_deployments)
    use_aoai_connection(
        validate_deployments,
        aoai_connection_id,
        custom_env="AZUREML_WORKSPACE_CONNECTION_ID_AOAI_COMPLETION",
    )

    git_clone = git_clone_component(
        git_repository=git_url,
    )
    use_automatic_compute(git_clone)

    crack_and_chunk = crack_and_chunk_component(
        input_data=git_clone.outputs.output_data,
        input_glob=data_source_glob,
        chunk_size=1024,
        data_source_url=data_source_url,
        document_path_replacement_regex=document_path_replacement_regex,
    )
    use_automatic_compute(crack_and_chunk)

    # QA Data Generation
    data_generation = data_generation_component(
        input_data=crack_and_chunk.outputs.output_chunks,
        deployment_validation=validate_deployments.outputs.output_data,
        llm_config=llm_completion_config,
        dataset_size=10,  # Number of QAs to be generated
    )
    use_automatic_compute(data_generation)
    use_aoai_connection(data_generation, aoai_connection_id)

    register_qa_data = register_qa_data_component(
        storage_uri=data_generation.outputs.output_data,
        asset_name=asset_name,
        register_output=True,
    )
    use_automatic_compute(register_qa_data)

    return {
        "qa_data": data_generation.outputs.output_data,
    }

The settings below show how the different git and data_source parameters can be set to processes only the AzureML documentation from the larger AzureDocs git repo, and ensure the source url for each document is processed to link to the publicly hosted URL instead of the git url.

In [ ]:
git_url = "https://github.com/MicrosoftDocs/azure-docs/"
data_source_url = "https://learn.microsoft.com/en-us/azure"
data_source_glob = "articles/machine-learning/**/*"
# This regex is used to remove the 'articles' folder from the source url put in each files metadata in the index.
document_path_replacement_regex = r'{"match_pattern": "(.*)/articles/(.*)(\\.[^.]+)$", "replacement_pattern": "\\1/\\2"}'
asset_name = "azure_docs_ml_qa_data"

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.entities import UserIdentityConfiguration

pipeline_job = qa_data_generation(
    git_url=git_url,
    data_source_glob=data_source_glob,
    data_source_url=data_source_url,
    document_path_replacement_regex=document_path_replacement_regex,
    aoai_connection_id=aoai_connection_id,
    llm_completion_config=llm_completion_config,
    asset_name=asset_name,
)

# By default AzureML Pipelines will reuse the output of previous component Runs when inputs have not changed.
# If you want to rerun the Pipeline every time each time so that any changes to upstream data sources are processed uncomment the below line.
# pipeline_job.settings.force_rerun = True # Rerun each time so that git_clone isn't cached, if intent is to ingest latest data.

### Submit Pipeline

**In case of any errors see [TROUBLESHOOT.md](../../TROUBLESHOOT.md).**

The output of each step in the pipeline can be inspected via the Workspace UI, click the link under 'Details Page' after running the below cell. 

In [ ]:
running_pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="qa_data_generation"
)
running_pipeline_job

### Review generated QA data

In [ ]:
import fsspec
import pandas as pd

qa_data = ml_client.data.get(f"{asset_name}-test-data", label="latest")
with fsspec.open(qa_data.path) as f:
    df = pd.read_json(f, lines=True)
df.head(2)

In [ ]:
for qa_type in ["TOPIC", "FACTUAL", "BOOLEAN"]:
    print(f"{qa_type} Question Answers:")
    for _, row in df[df["qaType"] == qa_type][:2].iterrows():
        print("Q:", row["question"])
        print("A:", row["answer"])
        print()

### Review token usage

In [ ]:
child_runs = ml_client.jobs.list(parent_job_name=running_pipeline_job.name)
child_runs = list(child_runs)
data_generation_run = child_runs[-1]

In [ ]:
from azureml.core import Run

run = Run.get(ws, data_generation_run.name)
metrics = run.get_metrics()

In [ ]:
print(f"Tokens used: {metrics['total_tokens']}")
print(f"Model used: {metrics['llm_model_name']}")

Given the token usage and the model you can compute cost using the pricing here: https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/.